In [1]:
!pip install -q openai pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00


In [2]:
import fitz
import json
from textwrap import dedent
from openai import OpenAI
from google.colab import userdata
import os

MODEL="gpt-4o"
client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"))

In [3]:

pdf_name = os.path.splitext(os.path.basename("j72.3_40.pdf"))[0]
doc = fitz.open("j72.3_40.pdf")
for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        pix = page.get_pixmap()
        output_path = f"{pdf_name}_page_{page_num + 1:03}.jpg"
        pix.save(output_path)

In [4]:
prompt = '''
    Transcribe the text.
    Only output the text and nothing else.
    Ignore the full width space (\u3000) in the text.
'''

In [5]:
import base64

def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode("utf-8")

In [15]:
def get_ocr(image_path):

    base64_image = encode_image(image_path)

    response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": dedent(prompt)
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                    }
                }
            ]
        }
    ],
    temperature=0.1,
    frequency_penalty=0.5,
    max_completion_tokens=1000,
    n=1,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "ocr_output",
            "schema": {
                  "type": "object",
                  "properties": {
                      "lines": {
                          "type": "array",
                          "items": {"type": "string"}
                      }
                  },
                  "required": ["lines"],
                  "additionalProperties": False
              },
              "strict": True
            },
        }
    )

    return response.choices[0].message

## TEST mit einem Bild

In [16]:
json_res = get_ocr("j72.3_40_page_002.jpg")
content_dict = json.loads(json_res.content)
print(content_dict["lines"])

['目次', 'はじめに', '第1章\u3000ジャーナリズムと権力', '1\u3000はじめに——語りについて 3', '2\u3000ジャーナリズムの仕事 8', '3\u3000ジャーナリズムの権力 12', '4\u3000ジャーナリストの専門性 18', '5\u3000ジャーナリストと情報源 24', '6\u3000報道機関とジャーナリストの関係 28', '7\u3000まとめ——語りとしてのニュース 32', '', '第2章\u3000ニュースの物語分析', '1\u3000はじめに——語りについて 35', '', '2\u3000ニュースの物語的構造——基本概念としての物語性と枠組み 42', '', '3\u3000ニューステクストへの言説分析（1）——表現及び表象をめぐる分析視角としての物語性と枠組み 47', '', '4\u3000ニューステクストへの言説分析（2）——表現及び表象をめぐる分析視角としての物語性と枠組み 50', '', '5\u3000まとめ——ニュースとは何か？ 57', '', '第3章 メディア・フレームと社会運動', '1 はじめに フレームとはなにか？61', '', '2 メディア・フレーム論の枠組み71', '', '3 社会運動とメディア・フレーム論との接点80']


In [17]:
import glob

In [18]:
images = glob.glob("*.jpg")
sorted_images = sorted(images, key=lambda x: int(x.split("_")[-1].split(".")[0]))

all_lines = []
for i in sorted_images:
    json_response = get_ocr(i)
    content_dict = json.loads(json_response.content)
    all_lines += content_dict["lines"]


print(all_lines)


['メディアの中の政治', '目次', '目次', 'はじめに', '第1章 ジャーナリズムと権力', '1 はじめに——語りについて 3', '2 ジャーナリズムの仕事 8', '3 ジャーナリズムの権力 12', '4 記者クラブとその問題 18', '5 記者クラブの機能(1)——情報管理機能 22', '6 記者クラブの機能(2)——排除機能 24', '7 記者クラブの機能(3)——記者同士の監視機能 26', '8 結論——ジャーナリストとの関係性を考えるために 28', '', '第2章 ニュースの物語分析', '', '1 はじめに——語りについて 35', '', '2 ニュースの物語的構成要素としての枠組み 42', '', '3 ニューステクストへの言説的枠組みとしての適用可能性 (1) ——ニューステクストへの言説的枠組みとしての適用可能性 (2) ——ニューステクストへの言説的枠組みとしての適用可能性 (3) ——ニューステクストへの言説的枠組みとしての適用可能性 (4) ——ニューステクストへの言説的枠組みとしての適用可能性 (5) ——ニューステクストへの言説的枠組みとしての適用可能性 (6) ——ニューステクストへの言説的枠組みとしての適用可能性 (7) ——ニューステクストへの言説的枠組みとしての適用可能性 (8) ——ニューステクストへの言説的枠組みとしての適用可能性 (9) ', '4 ニュース物語とメディアフレーム論との接点 ', '5 メディアフレーム論から見たニュース物語 ', '6 結論——メディアフレーム論から見た社会運動 ', '7 メディアフレームと社会運動 ', '8 メディアフレームと社会運動 ', '9 メディアフレームと社会運動 ', '10 メディアフレームと社会運動 ', '11 メディアフレームと社会運動 ', '12 メディアフレームと社会運動 ', '13 メディアフレームと社会運動 ', '第4章 世論調査という「権力」', 'はじめに', '1 近代日本における世論調査の登場 87', '2 戦後日本における世論調査の普及 93', '3 世論調査と同意の政治学 98', '4 世論調査と選挙報道 103', '5 むすび 107', '', '第5章 水俣病報道の物語', 'はじめに', '